# Control Run Analysis

Computation of the potential prognostis predictability (PPP) for metrics from the control simulation, using pre-built functions in project folder. 

**Important metrics:** <br>
- SH sea ice extent/volume
- regional sea ice extent/volume
- regional NPP, PCO2, SSS, SST
- gridpoint-level fields for sea ice concentration, SST, SSS, NPP, PCO2
- sea ice edge position (latitude at which sea ice concentration drops below 15%)

In [2]:
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")
warnings.filterwarnings("ignore", message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")

import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

In [5]:
rootdir = '/local/projects/so_predict/esm2m_froelicher/'
subdir = 'CTRL'

In [8]:
var = 'SST'
ds = xr.open_mfdataset(rootdir+subdir+'/'+var+'/'+'sst_*.nc')